In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/musicgenreclassification/features_3_sec.csv
/kaggle/input/musicgenreclassification/features_30_sec.csv
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00005.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00057.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00020.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00072.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00040.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00043.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00010.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00071.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00030.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00086.png
/kaggle/input/musicgenreclassification/images_waveplot/disco/disco00034.png
/kaggle/input/musicgenreclassification/images_w

In [2]:
import os
import torch
import torchaudio
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from torch import nn, optim
from tqdm import tqdm

2025-04-21 02:01:16.266138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745200876.457089      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745200876.509681      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
DATA_DIR = "/kaggle/input/musicgenreclassification/genres_original"
GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop',
          'jazz', 'metal', 'pop', 'reggae', 'rock']
LABEL_MAP = {genre: idx for idx, genre in enumerate(GENRES)}
SAMPLE_RATE = 16000
MAX_SECONDS = 3
MAX_LEN = SAMPLE_RATE * MAX_SECONDS

# Load Wav2Vec2
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
feature_extractor = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
feature_extractor.eval()

def load_audio_paths_and_labels(data_dir):
    '''Load data.'''
    paths = []
    labels = []
    for genre in GENRES:
        genre_dir = os.path.join(data_dir, genre)
        for fname in os.listdir(genre_dir):
            if fname.endswith(".wav"):
                if fname != "jazz.00054.wav":
                    paths.append(os.path.join(genre_dir, fname))
                    labels.append(LABEL_MAP[genre])
                else:
                    print(fname)
    return paths, labels
    
paths, labels = load_audio_paths_and_labels(DATA_DIR)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    paths, labels, test_size=0.2, random_state=42, stratify=labels)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


jazz.00054.wav


In [17]:
class Wav2VecGenreDataset(Dataset):
    '''Create Dataset.'''
    def __init__(self, file_paths, labels):
        self.paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        label = self.labels[idx]
        waveform, sr = torchaudio.load(path)
        waveform = waveform.mean(dim=0)
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLE_RATE)

        if waveform.shape[0] > MAX_LEN:
            waveform = waveform[:MAX_LEN]
        else:
            waveform = torch.nn.functional.pad(waveform, (0, MAX_LEN - waveform.shape[0]))

        inputs = processor(waveform, sampling_rate=SAMPLE_RATE, return_tensors="pt", padding=True)
        with torch.no_grad():
            features = feature_extractor(**inputs).last_hidden_state.mean(dim=1).squeeze()

        return features, label


train_ds = Wav2VecGenreDataset(train_paths, train_labels)
test_ds = Wav2VecGenreDataset(test_paths, test_labels)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=16, num_workers=2)

In [18]:
class GenreClassifier(nn.Module):
    '''Implement'''
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GenreClassifier(input_dim=768, num_classes=len(GENRES)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [19]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()            
        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader):.4f}")

Epoch 1: 100%|██████████| 50/50 [04:02<00:00,  4.84s/it]


Epoch 1 Loss: 2.1720


Epoch 2: 100%|██████████| 50/50 [03:49<00:00,  4.59s/it]


Epoch 2 Loss: 1.8470


Epoch 3: 100%|██████████| 50/50 [03:57<00:00,  4.75s/it]


...

Epoch 8 Loss: 1.2477


Epoch 9: 100%|██████████| 50/50 [03:50<00:00,  4.61s/it]


Epoch 9 Loss: 1.1934


Epoch 10: 100%|██████████| 50/50 [03:49<00:00,  4.60s/it]

Epoch 10 Loss: 1.1451


In [20]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"\n✅ Test Accuracy: {correct / total:.4f}")


✅ Test Accuracy: 0.6100
